
#### Phishing is a method of trying to gather personal information like login credentials or credit card information using deceptive e-mails or  websites.

#### Phishing websites are created to dupe unsuspecting users into thinking they are on a legitimate site. The criminals will spend a lot of time making the site seem as credible as possible and many sites will appear almost indistinguishable from the real thing 

#### So the essence of this code is to determine which sites are used as phishing sites from a given dataset using natural language processing and logistic regression

In [ ]:
# Importing some useful libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns  
import time 

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from nltk.tokenize import RegexpTokenizer  
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
import pickle

In [ ]:
# Loading the dataset
df = pd.read_csv(r"C:\Users\Olamzkid\Documents\Final Year Project\Dataset\phishing_site_urls.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

### About data
#### The dataset was gotten from kaggle.com which is an open source machine learning research center 
#### It consists of 549346 rows and 2 columns .The first column consist of links of website and the second column states whether the site is good or bad(phishing)

In [ ]:
df.isnull().sum()

In [ ]:
sns.countplot(x="Label",data=df)

### PREPROCESSING

#### Now we can Vectorize the URLs.We can gather words from the URLs using Tokenizer
#### Tokenization is the process of breaking down a piece of text into smaller units called tokens that makes it easier for the computer to understand and process so in this instance the tokenizer ignores numbers and special characters, extracting only sequences of alphabetic characters
### RegexpTokenizer
#### we are able to extract the tokens from string by using regular expression with RegexpTokenizer() method.
#### [A-Za-z]+ is used to match sequences of one or more alphabetic characters (both uppercase and lowercase).

In [ ]:
tokenizer = RegexpTokenizer(r'[A-Za-z]+')

In [ ]:
tokenizer.tokenize(df.URL[0]) # this will fetch all the words from the first URL

In [ ]:
# Tokenizing all the rows 
print('Getting words tokenized ...')
t0= time.perf_counter()
df['text_tokenized'] = df.URL.map(lambda t: tokenizer.tokenize(t))
t1 = time.perf_counter() - t0
print('Time taken',t1 ,'sec')

In [ ]:
df.sample(5)

### SnowballStemmer
#### Snowball is a small string processing language that gives the root words
#### it is essentially an algorithm for reducing words to their base or root form. In other words it shortens words i.e running to run jumping to jump and makes sure that the new words are treated the same as the old ones

In [ ]:
stemmer = SnowballStemmer("english") # choose a language

In [ ]:
# Getting all the stemmed words
print('Getting words stemmed ...')
t0= time.perf_counter()
df['text_stemmed'] = df['text_tokenized'].map(lambda l: [stemmer.stem(word) for word in l])
t1= time.perf_counter() - t0
print('Time taken',t1 ,'sec')

In [ ]:
df.sample(5)

In [ ]:
# Joining all the stemmmed words.
print('Get joiningwords ...')
t0= time.perf_counter()
df['text_sent'] = df['text_stemmed'].map(lambda l: ' '.join(l))
t1= time.perf_counter() - t0
print('Time taken',t1 ,'sec')

In [ ]:
bad_sites = df[df.Label == 'bad']
good_sites = df[df.Label == 'good']

In [ ]:
bad_sites.head()

In [ ]:
good_sites.head()

In [ ]:
df.head()

## Creating Model
### CountVectorizer- Convert a collection of text documents to a matrix of token counts

In [ ]:
cv = CountVectorizer()

In [ ]:
feature = cv.fit_transform(df.text_sent) #transform all text which we tokenize and stemed

In [ ]:
feature[:5].toarray() # convert sparse matrix into array to print transformed features

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
trainX, testX, trainY, testY = train_test_split(feature, df.Label)

### LogisticRegression
#### Logistic Regression is a Machine Learning classification algorithm that is used to predict the probability of a categorical dependent variable i.e (binary classification task) (yes, success, etc.) or 0 (no, failure, etc.)
#### Logistic regression is a statistical model commonly used for , where the outcome is one of two possible classes (e.g., spam or not spam, phishing or not phishing).

When applied to text classification tasks:

Tokenization: This process involves breaking down the text into individual units, such as words or tokens.

Vectorization: After tokenization, the text data is converted into numerical format (vectors) 

Logistic Regression: The logistic regression model then takes these vectors as input features. It learns the relationship between the features (vectorized words) and the binary outcome (e.g., phishing or not phishing).

Thats why im using logistic regression in this particular code

In [ ]:
from sklearn.linear_model import LogisticRegression


In [ ]:
lr = LogisticRegression()
lr.fit(trainX,trainY)

In [ ]:
lr.score(testX,testY)

#### Logistic Regression provide 96% accuracy,Now we will store the score in the dictionary so that we can find which model performs the best.


In [ ]:
Scores_ml = {}
Scores_ml['Logistic Regression'] = np.round(lr.score(testX,testY),2)

In [ ]:
# creating confusing matrix
print('Training Accuracy :',lr.score(trainX,trainY))
print('Testing Accuracy :',lr.score(testX,testY))
con_mat = pd.DataFrame(confusion_matrix(lr.predict(testX), testY),
            columns = ['Predicted:Bad', 'Predicted:Good'],
            index = ['Actual:Bad', 'Actual:Good'])


print('\nCLASSIFICATION REPORT\n')
print(classification_report(lr.predict(testX), testY,
                            target_names =['Bad','Good']))

print('\nCONFUSION MATRIX')
plt.figure(figsize= (6,4))
sns.heatmap(con_mat, annot = True,fmt='d',cmap="YlGnBu")

#### MultinomialNB

#### The multinomial Naive Bayes classifier is for text classification tasks, particularly when dealing with features that represent counts or frequencies of tokens which is exactly what ive been doing since 

#### The naive bayes classifier calculates the probability of the document belonging to each class using the features (word frequencies). It then predicts the class with the highest accuracy.

In [ ]:
from sklearn.naive_bayes import MultinomialNB 


In [ ]:
# create mnb object
mnb = MultinomialNB()

In [ ]:
mnb.fit(trainX,trainY)

In [ ]:
mnb.score(testX,testY)

#### MultinomialNB provide 95% accuracy,so we can store the score in the dictionary

In [ ]:
Scores_ml['MultinomialNB'] = np.round(mnb.score(testX,testY),2)

In [ ]:
print('Training Accuracy :',mnb.score(trainX,trainY))
print('Testing Accuracy :',mnb.score(testX,testY))
con_mat = pd.DataFrame(confusion_matrix(mnb.predict(testX), testY),
            columns = ['Predicted:Bad', 'Predicted:Good'],
            index = ['Actual:Bad', 'Actual:Good'])


print('\nCLASSIFICATION REPORT\n')
print(classification_report(mnb.predict(testX), testY,
                            target_names =['Bad','Good']))

print('\nCONFUSION MATRIX')
plt.figure(figsize= (6,4))
sns.heatmap(con_mat, annot = True,fmt='d',cmap="YlGnBu")

#### So, Logistic Regression is the best fit for this model, Now lets make sklearn pipeline using Logistic Regression

In [ ]:
pipeline_ls = make_pipeline(CountVectorizer(tokenizer = RegexpTokenizer(r'[A-Za-z]+').tokenize,stop_words='english'), LogisticRegression())

#### the code above works by :

#### It first tokenizes the input text using the specified regular expression tokenizer (RegexpTokenizer(r'[A-Za-z]+').tokenize).
#### It removes English stopwords.
#### It then converts the text data into a matrix of token counts (using CountVectorizer).
#### Finally, it applies logistic regression (LogisticRegression()) to classify the text based on the features generated by CountVectorizer.

In [ ]:
trainX, testX, trainY, testY = train_test_split(df.URL, df.Label)

In [ ]:
pipeline_ls.fit(trainX,trainY)

In [ ]:
pipeline_ls.score(testX,testY)

In [ ]:
print('Training Accuracy :',pipeline_ls.score(trainX,trainY))
print('Testing Accuracy :',pipeline_ls.score(testX,testY))
con_mat = pd.DataFrame(confusion_matrix(pipeline_ls.predict(testX), testY),
            columns = ['Predicted:Bad', 'Predicted:Good'],
            index = ['Actual:Bad', 'Actual:Good'])


print('\nCLASSIFICATION REPORT\n')
print(classification_report(pipeline_ls.predict(testX), testY,
                            target_names =['Bad','Good']))

print('\nCONFUSION MATRIX')
plt.figure(figsize= (6,4))
sns.heatmap(con_mat, annot = True,fmt='d',cmap="YlGnBu")

#### Now we dump the model in pickle format
#### pickle files are a convenient way to store and retrieve Python objects, including machine learning models and data, which helps streamline the development and deployment of machine learning applications

we can use it to store models for future use without having to retain the mode every time

In [ ]:
with open('phishingApp.pkl', 'wb') as model_file:
    pickle.dump(pipeline_ls, model_file)



In [ ]:
loaded_model = pickle.load(open('phishingApp.pkl', 'rb'))
result = loaded_model.score(testX,testY)
print(result)